<a href="https://colab.research.google.com/github/Vilmo18/Languages_Models_Without_Entity_Knowledge/blob/main/QA_with_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install --upgrade pyarrow

In [ ]:
!pip install -q transformers datasets
!pip install --upgrade pyarrow

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3fe09d08fcf04713ed33cf2134733cfa984b844f551176517dfbdbf3045c18d9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


## QA with entities

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
from datasets import load_dataset
dataset = load_dataset("squad")

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
anonymized_subset=load_from_disk("qa_good")
anonymized_subset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
id=14
anonymized_subset['train'][id]

{'id': '5733bed24776f4190066118c',
 'title': 'University_of_Notre_Dame',
 'context': 'The university is the major seat of ent113595 (albeit not its official headquarters, which are in ent185714). Its main seminary, ent21422, is located on the campus across St. ent148252 lake from ent211845. ent242011, the oldest building on campus and located near the shore of St. ent220024, houses undergraduate seminarians. Retired priests and brothers reside in ent10640 (a former retreat center), ent173298, as well as ent119769 near the Grotto. The university through ent63023 has ties to theologian ent144130. While not ent44296, ent46377 has praised writers from ent299290 and ent21422 created ent69726.',
 'question': 'Which prize did ent144130 create?',
 'answers': {'answer_start': [601], 'text': ['ent72089']}}

In [ ]:
id=14
dataset['train'][id]

{'id': '5733bed24776f4190066118c',
 'title': 'University_of_Notre_Dame',
 'context': 'The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.',
 'question': 'Which prize did Frederick Buechner create?',
 'answers': {'text': ['Buechner Prize for Preaching'], 'answer_start': [675]}}

In [ ]:
id=1
anonymized_subset['train'][2]

{'id': '5733be284776f41900661180',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a ent93676 character. Atop ent289216 gold dome is a golden statue of ent68466. Immediately in front of ent285133 and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to ent285133 is ent182288. Immediately behind the basilica is the Grotto, a ent78755 place of prayer and reflection. It is a replica of the grotto at ent249113, ent154860 where ent68466 reputedly appeared to ent223217 in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and ent209005), is a simple, modern stone statue of ent86434.',
 'question': 'ent21984 at ent59357 is beside to which structure?',
 'answers': {'answer_start': [254], 'text': ['ent285133']}}

In [ ]:
dataset['train'][id]

{'id': '56d3726559d6e414001463e9',
 'title': 'American_Idol',
 'context': 'Corey Clark was disqualified during the finals for having an undisclosed police record; however, he later alleged that he and Paula Abdul had an affair while on the show and that this contributed to his expulsion. Clark also claimed that Abdul gave him preferential treatment on the show due to their affair. The allegations were dismissed by Fox after an independent investigation. Two semi-finalists were also disqualified that year – Jaered Andrews for an arrest on an assault charge, and Frenchie Davis for having previously modelled for an adult website.',
 'question': 'Which contestant was removed from the competition for not revealing his police record?',
 'answers': {'text': ['Corey Clark'], 'answer_start': [0]}}

In [ ]:

train_ids_dataset1 = set(anonymized_subset['train']['id'])
validation_ids_dataset1 = set(anonymized_subset['validation']['id'])

# Filtrer le dataset2 pour ne garder que les éléments qui sont aussi dans dataset1
filtered_train_dataset2 = dataset['train'].filter(lambda example: example['id'] in train_ids_dataset1)
filtered_validation_dataset2 = dataset['validation'].filter(lambda example: example['id'] in validation_ids_dataset1)

# Créer le nouveau DatasetDict filtré
dataset = DatasetDict({
    'train': filtered_train_dataset2,
    'validation': filtered_validation_dataset2
})

# Afficher le résultat
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 75608
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 8663
    })
})


In [ ]:

from transformers import GPT2Tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

dataset['train'][3]

{'id': '5733be284776f41900661181',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is the Grotto at Notre Dame?',
 'answers': {'text': ['a Marian place of prayer and reflection'],
  'answer_start': [381]}}

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
        # stride=128,
        # return_overflowing_tokens=True,
        # #return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while idx < len(sequence_ids) and sequence_ids[idx] != 1: # Check if idx is within bounds
            idx += 1
        context_start = idx
        while idx < len(sequence_ids) and sequence_ids[idx] == 1: # Check if idx is within bounds
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1

            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import GPT2Tokenizer, GPT2ForQuestionAnswering, Trainer, TrainingArguments
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForQuestionAnswering.from_pretrained("openai-community/gpt2")

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [ ]:

from transformers import Trainer, DataCollatorWithPadding
from torch.utils.data import DataLoader  #
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq,DataCollatorForLanguageModeling

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
#model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    warmup_steps=1000,
    gradient_accumulation_steps=2,
    max_steps=5000,
    lr_scheduler_type="linear",
    logging_steps=10,
    learning_rate=3e-4,
    per_device_train_batch_size=8,  # 24 - 8
    weight_decay=0.01,
    #dataloader_num_workers=4,
    save_steps=0.1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    fp16=True,
    #report_to="tensorboard",
    eval_steps=0.1,
    seed=4201,
    #remove_unused_columns=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:

from transformers import Trainer, DataCollatorWithPadding
from torch.utils.data import DataLoader  #
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq,DataCollatorForLanguageModeling

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
#model.config.use_cache = False

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    warmup_steps=500,
    #gradient_accumulation_steps=2,
    max_steps=10000,
    lr_scheduler_type="linear",
    logging_steps=100,
    learning_rate=2e-4,
    per_device_train_batch_size=4,  # 24 - 8
    weight_decay=0.01,
    dataloader_num_workers=4,
    save_steps=0.1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    fp16=True,
    #report_to="tensorboard",
    eval_steps=0.1,
    seed=42,
    #remove_unused_columns=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: carre (carre-su). Use `wandb login --relogin` to force relogin


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
1000,2.347300,2.134022
2000,2.009200,1.842003
3000,1.730900,1.760921
4000,1.736800,1.640930
5000,1.707900,1.608805
6000,1.556900,1.543980
7000,1.564900,1.481084
8000,1.473900,1.428870
9000,1.429900,1.387852
10000,1.420100,1.366873


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid 

TrainOutput(global_step=10000, training_loss=1.7899933044433594, metrics={'train_runtime': 14816.3841, 'train_samples_per_second': 5.399, 'train_steps_per_second': 0.675, 'total_flos': 2.090374053888e+16, 'train_loss': 1.7899933044433594, 'epoch': 0.91324200913242})

In [ ]:
trainer.evaluate()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.3668725490570068,
 'eval_runtime': 192.4775,
 'eval_samples_per_second': 54.916,
 'eval_steps_per_second': 3.434,
 'epoch': 0.91324200913242}

## Test

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForQuestionAnswering
import torch

import torch

def generate_answer(question, context, model=model, tokenizer=tokenizer, top_k=1):
    # Encode inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

    # Move input tensors to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        # Get model outputs
        outputs = model(**inputs)

    # Compute span scores
    start_scores = outputs.start_logits[0].cpu().numpy()
    end_scores = outputs.end_logits[0].cpu().numpy()

    # Get top-k start and end positions
    start_indices = start_scores.argsort()[-top_k:][::-1]
    end_indices = end_scores.argsort()[-top_k:][::-1]


    answers = []
    for start_index in start_indices:
        for end_index in end_indices:
            if end_index >= start_index:
                answer_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index + 1])
                answer = tokenizer.convert_tokens_to_string(answer_tokens).strip()
                if answer:
                    answers.append((answer, start_scores[start_index] + end_scores[end_index]))

    answers = sorted(answers, key=lambda x: x[1], reverse=True)

    return answers[0][0] if answers else "No answer found"



context = "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. Constructed from 1887 to 1889 as the entrance arch to the 1889 World's Fair, it was initially criticized by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France and one of the most recognizable structures in the world."

questions = [
    "Who designed the Eiffel Tower?",
    "When was the Eiffel Tower built?",
    "Where is the Eiffel Tower located?",
    "What was the initial reaction to the Eiffel Tower?",
    "What is the Eiffel Tower made of?"
]

for question in questions:
    answer = generate_answer(question, context,model,tokenizer)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: Who designed the Eiffel Tower?
Answer: Gustave Eiffel

Question: When was the Eiffel Tower built?
Answer: 1887 to 1889

Question: Where is the Eiffel Tower located?
Answer: Paris, France

Question: What was the initial reaction to the Eiffel Tower?
Answer: criticized

Question: What is the Eiffel Tower made of?
Answer: a wrought-iron lattice tower



In [ ]:
idx=854
test_question = dataset['validation'][idx]['question']
test_context = dataset['validation'][idx]['context']
generated_answer = generate_answer(test_question, test_context,model,tokenizer)
print(f"Generated answer: {generated_answer}")


Generated answer: Ogród Saski


In [ ]:
dataset['validation'][idx]

{'id': '56dfa2c54a1a83140091ebf2',
 'title': 'Nikola_Tesla',
 'context': "In 1873, Tesla returned to his birthtown, Smiljan. Shortly after he arrived, Tesla contracted cholera; he was bedridden for nine months and was near death multiple times. Tesla's father, in a moment of despair, promised to send him to the best engineering school if he recovered from the illness (his father had originally wanted him to enter the priesthood).",
 'question': 'What disease did Tesla contract in 1873?',
 'answers': {'text': ['cholera', 'cholera', 'cholera'],
  'answer_start': [94, 94, 94]}}

## Evaluate

In [ ]:
!rm -rf ./results


In [ ]:
from rouge_score import rouge_scorer
from tqdm import tqdm
def evaluate_model_with_rouge(model, tokenizer, dataset):
    exact_match_total = 0
    f1_total = 0
    rouge_1_total = 0
    rouge_2_total = 0
    rouge_l_total = 0

    num_samples = len(dataset['validation'])
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    for i in tqdm(range(num_samples)):
        question = dataset['validation'][i]['question']
        context = dataset['validation'][i]['context']
        ground_truths = dataset['validation'][i]['answers']['text']

        generated_answer = generate_answer(question, context)

        exact_match_scores = [int(generated_answer.strip() == truth.strip()) for truth in ground_truths]
        exact_match_total += max(exact_match_scores)

        f1_scores = []
        rouge_1_scores = []
        rouge_2_scores = []
        rouge_l_scores = []

        for truth in ground_truths:
            generated_tokens = generated_answer.split()
            ground_truth_tokens = truth.split()

            common_tokens = set(generated_tokens) & set(ground_truth_tokens)
            precision = len(common_tokens) / len(generated_tokens) if generated_tokens else 0
            recall = len(common_tokens) / len(ground_truth_tokens) if ground_truth_tokens else 0
            f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            f1_scores.append(f1)

            rouge_scores = scorer.score(generated_answer, truth)
            rouge_1_scores.append(rouge_scores['rouge1'].fmeasure)
            rouge_2_scores.append(rouge_scores['rouge2'].fmeasure)
            rouge_l_scores.append(rouge_scores['rougeL'].fmeasure)

        f1_total += max(f1_scores)
        rouge_1_total += max(rouge_1_scores)
        rouge_2_total += max(rouge_2_scores)
        rouge_l_total += max(rouge_l_scores)

    avg_exact_match = exact_match_total / num_samples
    avg_f1_score = f1_total / num_samples
    avg_rouge_1 = rouge_1_total / num_samples
    avg_rouge_2 = rouge_2_total / num_samples
    avg_rouge_l = rouge_l_total / num_samples

    return avg_exact_match, avg_f1_score, avg_rouge_1, avg_rouge_2, avg_rouge_l

In [ ]:
avg_exact_match, avg_f1_score, avg_rouge_1, avg_rouge_2, avg_rouge_l = evaluate_model_with_rouge(model, tokenizer, dataset)
print(f"Exact Match: {avg_exact_match:.2f}")
print(f"F1 Score: {avg_f1_score:.2f}")
print(f"ROUGE-1: {avg_rouge_1:.2f}")
print(f"ROUGE-2: {avg_rouge_2:.2f}")
print(f"ROUGE-L: {avg_rouge_l:.2f}")

100%|██████████| 10570/10570 [11:02<00:00, 15.96it/s]

Exact Match: 0.63
F1 Score: 0.74
ROUGE-1: 0.75
ROUGE-2: 0.49
ROUGE-L: 0.75
